In [1]:
from langchain_community.llms import LlamaCpp

In [2]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/Users/reorganism/models/openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /Users/reorganism/models/openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = nurtureai_openhermes-2.5-mistral-7b-16k
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_

In [3]:
import os
import getpass

oo = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = oo
tt=os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tt

In [4]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")
os.environ["LANGCHAIN_PROJECT"] = "Plan-and-execute"

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [6]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4-turbo-preview")
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

ggml_metal_free: deallocating


In [7]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent_runnable, tools)


In [9]:
agent_executor.invoke(
    {"input": "What does it mean if I see a tiger in a dream?", "chat_history": []}
)

{'chat_history': [],
 'input': 'What does it mean if I see a tiger in a dream?',
 'agent_outcome': AgentFinish(return_values={'output': "Seeing a tiger in your dream can have various interpretations depending on the context of the dream, your emotions within the dream, and your personal life circumstances. However, some general meanings associated with tigers in dreams include:\n\n1. **Power and Strength**: Tigers are often seen as symbols of power, raw energy, and strength. Dreaming of a tiger may reflect your own personal power or a desire to embody more of these qualities.\n\n2. **Fear and Anxiety**: If the tiger in your dream is threatening or chasing you, it might symbolize fears or anxieties that you are facing in your waking life. The tiger could represent a situation or person that feels overpowering or daunting.\n\n3. **Aggression and Anger**: A tiger could also symbolize suppressed anger or aggression. It might indicate that you or someone around you is holding back feelings 

In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple, Annotated, TypedDict
import operator


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [11]:
from langchain_core.pydantic_v1 import BaseModel


class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

In [12]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}"""
)
planner = create_structured_output_runnable(
    Plan, ChatOpenAI(model="gpt-4-turbo-preview", temperature=0), planner_prompt
)

In [13]:
planner.invoke(
    {"objective": "What does it mean if I see a long lost friend in a dream?"}
)

Plan(steps=['Reflect on your feelings and emotions during the dream. Consider what your friend represented to you and how their appearance in the dream made you feel.', "Analyze the context in which your friend appeared in the dream. Consider the setting, any dialogue, and the overall situation to understand the dream's message.", 'Consider the current events in your life. Think about whether there are unresolved issues, desires for reconnection, or feelings of nostalgia that might have triggered the dream.', 'Research common dream interpretations related to seeing friends or loved ones to gain a broader understanding of potential meanings.', "Consult with a dream interpretation expert or psychologist if you're seeking a deeper analysis or if the dream has a significant impact on you emotionally.", 'Reflect on the insights gained from the analysis and how they relate to your personal growth or current life situation. Use this understanding to address any emotional needs or to appreciat

In [14]:
from langchain.chains.openai_functions import create_openai_fn_runnable


class Response(BaseModel):
    """Response to user."""

    response: str


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)


replanner = create_openai_fn_runnable(
    [Plan, Response],
    ChatOpenAI(model="gpt-4-turbo-preview", temperature=0),
    replanner_prompt,
)

In [15]:
async def execute_step(state: PlanExecute):
    task = state["plan"][0]
    agent_response = await agent_executor.ainvoke({"input": task, "chat_history": []})
    return {
        "past_steps": (task, agent_response["agent_outcome"].return_values["output"])
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"objective": state["input"]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output, Response):
        return {"response": output.response}
    else:
        return {"plan": output.steps}


def should_end(state: PlanExecute):
    if state["response"]:
        return True
    else:
        return False

In [16]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.set_entry_point("planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    {
        # If `tools`, then we call the tool node.
        True: END,
        False: "agent",
    },
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [17]:
from langchain_core.messages import HumanMessage

config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the 2024 Australia open winner?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

{'plan': ['Wait until the 2024 Australian Open concludes.', 'Identify the winner of the 2024 Australian Open.', "Research the winner's biography to find their hometown."]}
{'past_steps': ('Wait until the 2024 Australian Open concludes.', "I can't wait for events or perform actions in future dates. However, I can provide you with information about the 2024 Australian Open or help with related queries. Do you have any specific questions or need information on how to follow the event?")}


KeyError: 'response'